In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/meruozhu/Downloads/MP_data/MP_codes/MP')
from DPM.task_free_continual_learning.method import Task_free_continual_learning
from DPM.task_free_continual_learning.sampler import Sampler

/Users/meruozhu/opt/anaconda3/envs/pmGrad/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import time
import pandas as pd
import numpy as np
from scipy.stats import percentileofscore
from Data.data import Data
from edbn.Utils.LogFile import LogFile
import edbn.Predictions.setting as setting
from edbn import Methods
from collections import deque
from itertools import islice
from collections import OrderedDict
from PrefixTreeCDD.PrefixTreeClass import PrefixTree
import PrefixTreeCDD.settings as settings
from PrefixTreeCDD.CDD import Window
from skmultiflow.drift_detection import ADWIN, PageHinkley
import math
from numpy import log as ln
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import utils as ku
import numpy.random as rn

### Data

In [36]:
df = pd.read_csv('/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/Helpdesk.csv')
# Keep only the first 300 rows
df = df[:300]

# Adding column for first 150 rows
df.loc[df.index[:150], 'new_col'] = df['event'].apply(lambda x: 1 if x == 'Assign seriousness' else 0)

# Adding column for last 150 rows
df.loc[df.index[-150:], 'new_col'] = df['event'].apply(lambda x: 0 if x == 'Assign seriousness' else 1)

# Save the new DataFrame to a CSV file
df.to_csv('/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/Helpdesk_mini.csv', index=False)

### get basic model

In [4]:
file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/Helpdesk_mini.csv'
data = pd.read_csv(file, low_memory=False)
timeformat = "%Y-%m-%d %H:%M:%S"
numEvents = data.shape[0]
print("Num events is {}".format(numEvents))

dataName = 'Helpdesk_mini'
log = LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False)
d = Data(dataName,
            LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False))
d.logfile.keep_attributes(['event', 'role', 'completeTime'])
m = Methods.get_prediction_method("SDL")
s = setting.STANDARD
trainPerc = 0.5
s.train_percentage = trainPerc * 100
# # #
d.prepare(s)

Num events is 300
PREPARE
CONVERT
PREPROCESSING: Converting event
PREPROCESSING: Converting role
PREPROCESSING: Converting case
K-CONTEXT
Create k-context: 10
SPLIT TRAIN-TEST
Train: 150
Test: 150


In [5]:
file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/Helpdesk_mini.csv'
data = pd.read_csv(file, low_memory=False)
timeformat = "%Y-%m-%d %H:%M:%S"
numEvents = data.shape[0]
print("Num events is {}".format(numEvents))

dataName = 'Helpdesk_mini'
d = Data(dataName,
            LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False))
d.logfile.keep_attributes(['event', 'role', 'completeTime'])
m = Methods.get_prediction_method("SDL")
s = setting.STANDARD
trainPerc = 0.5
s.train_percentage = trainPerc * 100
# # #
d.prepare(s)
# d.create_batch("normal", timeformat)
is_written = 0

start_time = time.time()
# print(d.train.contextdata)
print("Test Context Data")
print(d.test_orig.contextdata)
basic_model = m.train(d.train)

Num events is 300
PREPARE
CONVERT
PREPROCESSING: Converting event
PREPROCESSING: Converting role
PREPROCESSING: Converting case
K-CONTEXT
Create k-context: 10
SPLIT TRAIN-TEST
Train: 150
Test: 150
Test Context Data
     level_0  index  event_Prev9  role_Prev9  completeTime_Prev9  case_Prev9  \
0        150    176            0           0                   0          53   
1        151    177            0           0                   0          53   
2        152    252            0           0                   0          73   
3        153    108            0           0                   0          31   
4        154    109            0           0                   0          31   
..       ...    ...          ...         ...                 ...         ...   
145      295    225            0           0                   0          68   
146      296    226            0           0                   0          68   
147      297    167            0           0                   0 

2023-05-13 13:24:45.378464: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-13 13:24:45.379476: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
event_Prev0 (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
event_Prev1 (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
event_Prev2 (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
event_Prev3 (InputLayer)        [(None, 1)]          0                                            
______________________________________________________________________________________________

### Sampler

In [146]:
alpha=1.0, 

ntasks=2
dim=4
discriminator_offset=0.01
distribution_offset=0.01
uniform_width=1.25
nsamples=500
ntrain=40
ntest=2
# seed
rn.seed(512)
stime=time.time()
tasks={}
        
# sample points for four tasks related to four quadrants
for q in range(ntasks):
    #print("quadrant {0}".format(q))
    # step 1 sample points in quadrant uniformly from [0:1]
    if q == 0: # 1th task
        # slice the dataframe to keep all columns except the last one
        # xs=data.iloc[:, :-1]
        # ys=data.iloc[:, -1]
        xs=rn.uniform(distribution_offset,uniform_width,nsamples)
        # print('xs',xs,len(xs))
        ys=rn.uniform(distribution_offset,uniform_width,nsamples)
    elif q == 1: # 2nd task
        xs=rn.uniform(-distribution_offset,-uniform_width,nsamples)
        ys=rn.uniform(distribution_offset,uniform_width,nsamples)
    elif q == 2: # 3th quadrant
        xs=rn.uniform(-distribution_offset,-uniform_width,nsamples)
        ys=rn.uniform(-distribution_offset,-uniform_width,nsamples)
    else: #4th quadrant
        xs=rn.uniform(distribution_offset,uniform_width,nsamples)
        ys=rn.uniform(-distribution_offset,-uniform_width,nsamples)
    samples=[]
    for i in range(len(xs)):
        sample=[xs[i],ys[i]]
        for j in range(dim-2): #add noise gausians in other dimensions
            sample.append(rn.uniform(-uniform_width,uniform_width))
            #print(sample)
        samples.append(sample)
    # print('samples',samples)
    # print('arrays',np.asarray(samples))
    tasks[q]=np.asarray(samples)

In [150]:
samples[0]

[-0.42015664278236897,
 0.8506855260539721,
 -0.5302694932323413,
 0.37004551164606125]

In [148]:

inputs={}
labels={}

# Step 3: Sample correct distribution given certain alpha
for q in range(ntasks):
    pos_inputs=[]
    neg_inputs=[]
    other_tasks=list(range(ntasks))
    del other_tasks[q]
    while not (len(pos_inputs) == ntrain/2 and len(neg_inputs) == ntrain/2): 
        if rn.binomial(1,alpha): #1 if original task, 0 if data sampled from 1 of the others.
            q_temp=q
        else: # pick from random other quadrant
            q_temp=rn.choice(other_tasks)
        sample=tasks[q_temp][0]
        # samples are popped from the distribution to avoid them to be reused.
        tasks[q_temp]=tasks[q_temp][1:]
        if np.sqrt(np.sum(sample**2))>1+discriminator_offset and len(neg_inputs) < ntrain/2:
            neg_inputs.append(sample) 
        #elif np.sqrt(np.sum(sample**2))<1:
        elif np.sqrt(np.sum(sample**2))<1-discriminator_offset and len(pos_inputs) < ntrain/2:
            pos_inputs.append(sample) 
        else: #discard samples on border as task becomes too hard 
            pass
    pos_inputs=pos_inputs[:int(ntrain/2)]
    # print(pos_inputs)
    neg_inputs=neg_inputs[:int(ntrain/2)]
    # print(neg_inputs)
    inputs[q]=pos_inputs+neg_inputs
    rn.shuffle(inputs[q])
    labels[q]=[]
    for sample in inputs[q]:
        if np.sqrt(np.sum(sample**2))>1+discriminator_offset:
            labels[q].append(0) 
        elif np.sqrt(np.sum(sample**2))<1-discriminator_offset:
            labels[q].append(1) 
        else: #discard samples on border as task becomes too hard 
            pass
    #if verbose: plt.plot(sample[0],sample[1],icon[q])
    print('{0}:proportion positive/total={1}'.format(q,float(sum(labels[q]))/len(labels[q])))

0:proportion positive/total=0.5
1:proportion positive/total=0.5


In [171]:
labels[1][0]

0

In [170]:
inputs[0][0]

array([ 1.00628232,  0.45844909,  1.08771111, -0.14832103])

In [143]:
test_inputs={}
test_labels={}
for q in range(ntasks):
    test_inputs_pos=[]
    test_inputs_neg=[]
    while len(test_inputs_pos)<ntest/2 or len(test_inputs_neg)<ntest/2:
        sample=tasks[q][0]
        tasks[q]=tasks[q][1:]
        # map sample to 2D plane for visualizations
        if len(sample)>2: sample[2:]=0
        if np.sqrt(np.sum(sample**2)) > 1+discriminator_offset:
            test_inputs_neg.append(sample)
            #test_inputs[q].append(sample)
            #test_labels[q].append(0)
        #elif np.sqrt(np.sum(sample**2)) < 1:
        elif np.sqrt(np.sum(sample**2)) < 1-discriminator_offset:
            test_inputs_pos.append(sample)
            #test_inputs[q].append(sample)
            #test_labels[q].append(1)
        else:
            pass
    test_inputs[q]=np.asarray(test_inputs_pos[:int(ntest/2)]+test_inputs_neg[:int(ntest/2)])
    test_labels[q]=[1]*int(ntest/2)+[0]*int(ntest/2)
    #print test_inputs[q].shape
# Check distribution
# for q in range(ntasks):
#     positive_points=[test_inputs[q][i] for i in range(len(test_inputs[q])) if test_labels[q][i]==1]
#     negative_points=[test_inputs[q][i] for i in range(len(test_inputs[q])) if test_labels[q][i]==0]

#     if verbose: plt.scatter([p[0] for p in positive_points],[p[1] for p in positive_points],color='blue')
#     if verbose: plt.scatter([p[0] for p in negative_points],[p[1] for p in negative_points],color='red')
# if verbose: plt.show()
print("sampling data duration: ",time.time()-stime)

print(type(tasks),len(tasks[0]))
print(tasks.keys)
print(type(inputs),inputs[0],len(inputs[0]))
print(type(labels),labels[0],len(labels[0]))
# print(test_inputs)
# print(test_labels)

sampling data duration:  446.4313819408417
<class 'dict'> 199
<built-in method keys of dict object at 0x7f8dac1d6780>
<class 'dict'> [array([ 1.00628232,  0.45844909,  1.08771111, -0.14832103]), array([ 0.15323392,  0.97929753, -1.22531805,  1.18517765]), array([0.11512547, 0.55350968, 0.93751561, 1.08646026]), array([ 0.77614699,  0.08418841, -1.19811774,  1.23385797]), array([ 0.33833774,  0.52128317,  0.39715614, -0.36154088]), array([ 0.07006794,  0.71839165, -0.17742743, -0.16690964]), array([ 0.55793649,  1.04764909,  0.50534306, -0.08130052]), array([ 0.49093612,  0.73437225, -0.76831385, -0.78087616]), array([ 0.53147875,  0.94836833, -0.44182229,  0.73560696]), array([ 0.36582721,  0.71278102,  0.35909352, -0.26635083]), array([0.59866706, 0.11612203, 0.77091497, 0.08967942]), array([ 0.14304687,  0.097595  , -0.62552362,  0.89825062]), array([0.04408643, 0.3862239 , 1.07580303, 0.03786426]), array([ 0.01475308,  0.0130378 ,  0.1076325 , -0.82526596]), array([ 0.27280179,  0.8

In [145]:
type(samples)

list

In [78]:
df=d.train.get_data()
# Adding column for first 150 rows
df.loc[df.index[:150], 'new_col'] = df['event'].apply(lambda x: 1 if x == 'Assign seriousness' else 0)

# Adding column for last 150 rows
df.loc[df.index[-150:], 'new_col'] = df['event'].apply(lambda x: 0 if x == 'Assign seriousness' else 1)
x = df.drop('new_col', axis=1)
x = x.values.tolist()


In [32]:
def transform_data(log, columns):
    num_activities = len(log.values[log.activity]) + 1

    col_num_vals = {}
    for col in columns:
        if col == log.activity:
            col_num_vals[col] = num_activities
        else:
            col_num_vals[col] = log.contextdata[col].max() + 2

    inputs = []
    for _ in range(len(columns) * log.k - len(log.ignoreHistoryAttributes) * log.k):
        inputs.append([])
    outputs = []
    for row in log.contextdata.iterrows():
        row = row[1]
        i = 0
        for attr in columns:
            if attr not in log.ignoreHistoryAttributes:
                for k in range(log.k):
                    inputs[i].append(row[attr + "_Prev%i" % k])
                    i += 1
        outputs.append(row[log.activity])

    outputs = ku.to_categorical(outputs, num_activities)
    for i in range(len(inputs)):
        inputs[i] = np.array(inputs[i])
    return inputs, outputs, col_num_vals

In [61]:
x, y, vals= transform_data(d.train, [a for a in d.train.attributes() if a != d.train.time and a != d.train.trace])

In [86]:
x

[array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 6, 0, 0, 1, 6, 0, 1, 6, 0, 1,
        6, 0, 0, 0, 1, 1, 0, 1, 1, 6, 0, 1, 0, 1, 6, 0, 1, 6, 1, 6, 0, 1,
        6, 0, 6, 1, 1, 6, 5, 5, 6, 1, 6, 0, 1, 6, 1, 6, 1, 0, 0, 0, 1, 6,
        0, 0, 0, 1, 1, 6, 0, 1, 0, 1, 6, 0, 1, 1, 6, 0, 0, 1, 6, 1, 6, 0,
        0, 6, 0, 1, 6, 0, 1, 6, 7, 0, 0, 1, 6, 1, 0, 1, 6, 0, 1, 6, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 6, 0, 0, 1, 6, 5, 6, 1, 5, 2, 5, 2, 5, 5, 1,
        1, 6, 1, 6, 1, 1, 0, 1, 6, 5, 1, 6, 0, 1, 6, 0, 1, 6]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
        1, 0, 1, 0, 1, 1, 6, 6, 5, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
        0, 5, 0, 0, 1, 0, 0, 1, 6, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 6, 1, 0, 6, 5, 6, 5, 6, 6, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 6, 1, 1, 0, 0

In [92]:
basic_model.predict(x)[0]

array([0.08845771, 0.25655437, 0.08241332, 0.09202303, 0.09101397,
       0.13982287, 0.16602792, 0.08368681], dtype=float32)

In [193]:

for i in range(len(d.test_orig.get_data())):
    sample = []
    for j in range(len(x)):
        sample.append(np.array([x[j][i]]))
        #sample.append(x[j][i])
                 

In [194]:
sample

[array([6]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([6]),
 array([6]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0])]

In [192]:
np.argmax(basic_model.predict(sample))

ValueError: Error when checking input: expected event_Prev0 to have 2 dimensions, but got array with shape ()

In [97]:
def predict_single_sample(model, log):
    inputs, expected, _ = transform_data(log, [a for a in log.attributes() if a != log.time and a != log.trace])
    predictions = model.predict(inputs)
    predict_vals = np.argmax(predictions, axis=1)
    return predict_vals
predict_single_sample(basic_model, d.train)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [45]:
def test(model, log):
    inputs, expected, _ = transform_data(log, [a for a in log.attributes() if a != log.time and a != log.trace])
    predictions = model.predict(inputs)
    predict_vals = np.argmax(predictions, axis=1)
    predict_probs = predictions[np.arange(predictions.shape[0]), predict_vals]
    expected_vals = np.argmax(expected, axis=1)
    expected_probs = predictions[np.arange(predictions.shape[0]), expected_vals]
    result = zip(expected_vals, predict_vals, predict_probs, expected_probs)
    return result

### task free

In [6]:
learning_object=Task_free_continual_learning(verbose=False,
                                                        seed=123,
                                                        dev='cpu',
                                                        dim=4,
                                                        hidden_units=100,
                                                        learning_rate=0.005,
                                                        ntasks=2,
                                                        gradient_steps=5,
                                                        loss_window_length=5,
                                                        loss_window_mean_threshold=0.2,
                                                        loss_window_variance_threshold=0.1,                                                         
                                                        MAS_weight=0.5,
                                                        recent_buffer_size=20,
                                                        hard_buffer_size=5)


In [27]:
learning_object.method(Sampler(),use_hard_buffer=True,continual_learning=True)

0:proportion positive/total=0.5
1:proportion positive/total=0.5
sampling data duration:  0.06032299995422363
tensor([[ 0.9366,  0.0674],
        [ 1.1033, -0.0679],
        [ 1.0133,  0.0115],
        [ 1.0774, -0.0627],
        [-0.0017,  1.0195],
        [-0.0291,  1.0172],
        [ 0.7884,  0.1926],
        [ 0.7779,  0.1918],
        [ 0.8406,  0.1191],
        [ 0.1016,  0.9207],
        [ 0.0942,  0.9216],
        [ 0.8361,  0.1759],
        [ 0.7621,  0.2222],
        [-0.0267,  1.0339],
        [ 1.0613, -0.0415],
        [-0.0106,  1.0170],
        [ 1.0689, -0.0546],
        [ 1.0953, -0.0666],
        [ 0.0243,  0.9706],
        [ 0.8838,  0.1358]], grad_fn=<MmBackward>)
(20, 2)
tensor([[ 0.9473,  0.0685],
        [-0.0768,  1.0691],
        [ 1.1499, -0.1261],
        [ 0.1593,  0.8337],
        [ 0.9288,  0.0392],
        [ 0.0564,  0.9339],
        [ 0.0657,  0.9394],
        [ 1.0205, -0.0458],
        [ 0.0235,  0.9461],
        [ 0.0083,  0.9552],
        [-0.1178,  1

([1.0, 1.0, 1.0, 1.0],
 [0.6324362, 0.6523496, 0.9522458, 0.9746931],
 [0, 0, 0, 0],
 [0.0, 0.0003965425, 0.18013982, 0.13661651],
 {0: [1.0, 1.0, 1.0, 1.0], 1: [0.5, 1.0, 0.5, 0.5]})

In [116]:
data.tasks[0].size

952

In [118]:
data.tasks[0][0]

array([ 0.14288676,  0.80488097,  0.34772433, -0.61486341])

In [172]:
data.inputs[0]

AttributeError: 'DataFrame' object has no attribute 'inputs'

In [125]:
len(data.labels[1])

40